In [2]:
%matplotlib inline
import graphlab
import graphlab.aggregate as agg
import numpy as np
import matplotlib.pyplot as plt  
import string
from nltk import PorterStemmer
from nltk.tokenize import RegexpTokenizer
from bs4 import BeautifulSoup
from nltk.corpus import stopwords

stemmer = PorterStemmer()

plt.style.use('ggplot')

In [103]:
sf = graphlab.SFrame('./data/boardgames-no-comments')
sf.remove_columns(['age', 'average', 'bayesaverage', 'maxplayers', 'maxplaytime', 'minplayers', 'minplaytime', 'owned', 'playingtime', 'stddev', 'trading', 'wanting', 'wishing', 'yearpublished'])
sf = sf[sf['usersrated'] > 500]# sf = sf['Expansion for Base-game' not in sf['categories']]
sf['categories'] = sf[['categories']].apply(lambda x: {w: 1 for w in x['categories']})
sf['mechanics'] = sf[['mechanics']].apply(lambda x: {w: 1 for w in x['mechanics']})
sf['subdomains'] = sf[['subdomains']].apply(lambda x: {w: 1 for w in x['subdomains']})
sf = sf[sf.apply(lambda x: 'Expansion for Base-game' not in x['categories'])]
len(sf)

2112

In [104]:
categories = graphlab.SFrame()
categories['name'] = set().union(*sf[['categories']].unstack('categories')[0]['List of categories'])

In [105]:
subdomains = graphlab.SFrame()
subdomains['name'] = set().union(*sf[['subdomains']].unstack('subdomains')[0]['List of subdomains'])

In [106]:
mechanics = graphlab.SFrame()
mechanics['name'] = set().union(*sf[['mechanics']].unstack('mechanics')[0]['List of mechanics'])
mechanics.head()

name
Card Drafting
Hand Management
Grid Movement
Modular Board
Line Drawing
Acting
Time Track
Rock-Paper-Scissors
Memory
Trick-taking


In [107]:
model = graphlab.topic_model.create(sf['categories'], num_topics=10, num_iterations=100)

PROGRESS: Learning a topic model
PROGRESS:        Number of documents      2112
PROGRESS:            Vocabulary size        82
PROGRESS:    Running collapsed Gibbs sampling
PROGRESS: +-----------+---------------+----------------+-----------------+
PROGRESS: | Iteration | Elapsed Time  | Tokens/Second  | Est. Perplexity |
PROGRESS: +-----------+---------------+----------------+-----------------+
PROGRESS: | 10        | 15.632ms      | 6.01534e+006   | 0               |
PROGRESS: | 20        | 31.281ms      | 2.99847e+006   | 0               |
PROGRESS: | 30        | 45.938ms      | 6.01534e+006   | 0               |
PROGRESS: | 40        | 59.625ms      | 6.02149e+006   | 0               |
PROGRESS: | 50        | 73.311ms      | 6.02766e+006   | 0               |
PROGRESS: | 60        | 86.999ms      | 3.00613e+006   | 0               |
PROGRESS: | 70        | 99.707ms      | 6.02149e+006   | 0               |
PROGRESS: | 80        | 113.394ms     | 3.00767e+006   | 0               |
PR

In [108]:
sf['category_topic']= model.predict(sf['categories'])

In [109]:
sf.select_columns(['name', 'description', 'categories', 'category_topic']).save('./topics.csv', 'csv')

In [112]:
model.get_topics().print_rows(500)

+-------+---------------------------+-----------------+
| topic |            word           |      score      |
+-------+---------------------------+-----------------+
|   0   |         Party Game        |  0.237943737441 |
|   0   |          Bluffing         |  0.202779638312 |
|   0   |           Humor           |  0.16259209645  |
|   0   |     Territory Building    |  0.137474882786 |
|   0   |           Trivia          | 0.0504018754186 |
|   1   |          Medieval         |  0.223847212663 |
|   1   |       City Building       |  0.170509291122 |
|   1   |          Nautical         |  0.094803854095 |
|   1   |           Racing          | 0.0775980729525 |
|   1   |        Renaissance        | 0.0707157604955 |
|   2   |          Economic         |  0.433091939547 |
|   2   |        Negotiation        |  0.141845088161 |
|   2   |        Civilization       | 0.0883186397985 |
|   2   |       Transportation      | 0.0662783375315 |
|   2   |  Industry / Manufacturing | 0.06627833